### Preparacion de los datos, para el archivo csv ya unido y cuando ya se han eliminado todas las columnas que no sirven, y todos los datos que no son Capital y GBA

In [1]:
import pandas as pd
import numpy as np

In [ ]:
propiedades = pd.read_csv('properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [ ]:
#Reemplazo todos los tipos de Palermo por uno unico

propiedades.loc[propiedades.place_name.str.contains("Palermo", na = False),'place_name'] = 'Palermo'
propiedades.loc[propiedades.place_name.str.contains("Palermo", na = False),'place_name'].value_counts()


In [ ]:
#Reemplazo todos los tipos de Lanus por uno unico

propiedades.loc[(propiedades.place_name.str.contains("Lanus", na = False)) | (propiedades.place_name.str.contains("Lanús", na = False)),'place_name'] = 'Lanús'
propiedades.loc[propiedades.place_name.str.contains("Lanús", na = False),'place_name'].value_counts()


In [2]:
def unificar_superficie(fila):
    s3,s2,s1 = fila
    if (s3 and not np.isnan(s3)):
        return s3
    if (s2 and not np.isnan(s2)):
        return s2
    if (s1 and not np.isnan(s1)):
        return s1
    return np.nan


In [ ]:
#Unifico las superficies que estan en tres columnas

propiedades.loc[:,'superficie'] = propiedades.loc[:,['surface_covered_in_m2','surface_in_m2','surface_total_in_m2']].apply(unificar_superficie, axis = 1)
propiedades.drop(['surface_covered_in_m2','surface_in_m2','surface_total_in_m2'], axis = 1, inplace = True)


In [4]:
def calcularPrecioM2(fila):
    """Recibe una lista con precio total, superficie y precio por metro cuadrado,
    y calcula este ultimo a partir de los anteriores, cuando es posible"""
    precio_total, precio_m2, superficie = fila
    if (np.isnan(precio_m2)):
        if (superficie and not np.isnan(superficie)):
            return precio_total/superficie
    return precio_m2


In [ ]:
#Calculo el precio por metro cuadrado a partir del precio total y la superficie

propiedades.loc[:,'price_usd_per_m2'] = propiedades.loc[:,['price_aprox_usd','price_usd_per_m2', 'superficie']].\
            apply(calcularPrecioM2,axis = 1)


In [6]:
def obtener_anio(fecha):
    return int(fecha.split('-')[0])

def obtener_mes(fecha):
       return int(fecha.split('-')[1])

In [ ]:
#Calculo del anio y mes de publicacion

propiedades.loc[:,'Year'] = propiedades.loc[:,'fecha_de_publicacion'].apply(obtener_anio)
propiedades.loc[:,'Month'] = propiedades.loc[:,'fecha_de_publicacion'].apply(obtener_mes)

In [ ]:
#Calculo del anio y mes de creacion

propiedades.loc[:,'Year_created'] = propiedades.loc[:,'created_on'].apply(obtener_anio)
propiedades.loc[:,'Month_created'] = propiedades.loc[:,'created_on'].apply(obtener_mes)


In [ ]:
def distancia_entre_fechas(fila):
    year, year_created, month, month_created = fila
    return (month - month_created) + 12 * (year - year_created)
    

In [ ]:
#Calculo la cantidad de meses que duro la publicacion

propiedades.loc[:, 'cantidad_meses_publicado'] = propiedades.loc[:,['Year', 'Year_created', 'Month', 'Month_created']]\
        .apply(distancia_entre_fechas, axis = 1)

In [7]:
def convertir_expensas(fila):
    """Convierto las expensas a float, y si no se puede devuelve nan"""
    try:
        return float(fila)
    except:
        return np.nan


In [ ]:
#Convierto las expensas a float

propiedades.loc[:,'expenses'] = propiedades.loc[:,'expenses'].apply(convertir_expensas)